In [ ]:
from google.colab import drive
import pandas as pd
import os
import csv
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls

drive  sample_data


In [ ]:
def list_files(directory):
    files = os.listdir(directory)
    return files

# Replace 'directory_path' with the path of the directory you want to list the files of
directory_path = './drive/MyDrive/raw_data'
files = list_files(directory_path)

In [ ]:
import csv

# Define headers
headers = ["query", "lang1", "lang2"]

# Define the file path for the new CSV file
new_csv_file_path = "./drive/MyDrive/preprocessed.csv"

# Open the new CSV file in write mode
with open(new_csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write headers to the CSV file
    writer.writerow(headers)

print(f"New CSV file '{new_csv_file_path}' has been created with headers only.")

New CSV file './drive/MyDrive/preprocessed.csv' has been created with headers only.


In [ ]:
import os
import pandas as pd

def process_language_files(directory_path: str, files: list[str], lang1: str, lang2: str, csv_path: str) -> None:
    all_data = []

    for file in files:
        file_path = os.path.join(directory_path, file)
        print(f"Processing {file_path}")
        try:
            df = pd.read_csv(file_path, on_bad_lines='skip')
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

        query = df.columns.values[0].split(' -')[0]
        lang1_index = None
        lang2_index = None

        languages = df.iloc[:, 0]

        for i, language in enumerate(languages):
            if language == lang1:
                lang1_index = i
            elif language == lang2:
                lang2_index = i
        if lang1_index is None or lang2_index is None:
            try:
                query, lang1_text, lang2_text = if_is_broken(file_path, lang1, lang2)
                all_data.append([query, lang1_text, lang2_text])
            except ValueError as e:
                print(f"Error occurred while processing file: {e}")
                continue
            continue

        lang1_data = df.iloc[lang1_index, 1:]
        lang2_data = df.iloc[lang2_index, 1:]
        lang1_text = ""
        lang2_text = ""
        for data in lang1_data:
            if not isinstance(data, str) or pd.isna(data):
                continue
            lang1_text += data
        for data in lang2_data:
            if not isinstance(data, str) or pd.isna(data):
                continue
            lang2_text += data

        all_data.append([query, lang1_text, lang2_text])

    all_data_df = pd.DataFrame(all_data, columns=['query', 'lang1', 'lang2'])

    try:
        all_data_df.to_csv(csv_path, mode='a', index=False, header=not os.path.exists(csv_path))
        print(f"Data written to {csv_path}")
    except Exception as e:
        print(f"Error occurred while writing to file: {e}")


In [ ]:
def if_is_broken(file_path: str, lang1: str, lang2: str) -> list[str]:
   with open(file_path, newline='') as csvfile:
    query, lang1_text, lang2_text = "", "", ""

    reader = csv.reader(csvfile)

    # Iterate over each row in the CSV file
    for index,row in enumerate(reader):
        # Access the row here
        if index == 0:
            query = row[0].split(' -')[0]
        else:
            if row[0] == lang1:
                lang1_text = ''.join(row[1:])
            elif row[0] == lang2:
                lang2_text = ''.join(row[1:])
    if len(lang1_text) <= 1 or len(lang2_text) <= 1:
         raise ValueError("Text data is missing for one or both of the provided languages.")
    return [query, lang1_text, lang2_text]

In [ ]:
process_language_files(directory_path,files,"Java", "Python", new_csv_file_path)